In [1]:
import numpy as np

import chugunov_indicator as chug
from chugunov_indicator.default_data import DefaultScreeningData

import pynucastro as pyna
import yt

rng = np.random.default_rng(0)

To see how effective the interpolation and prediction procedure in `interpolation.py` is, we can test it on a different snapshot of double detonation data.

In [2]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt17526/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-01 22:50:03,484 Parameters: current_time              = 0.800001438968278
yt : [INFO     ] 2024-10-01 22:50:03,485 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-01 22:50:03,486 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-01 22:50:03,487 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


First, we take a sample of $10,000$ points in the data.

In [3]:
total = 10000

In [4]:
choices = rng.choice(np.arange(len(ad["density"])), total)
choices

array([2936273, 2198730, 1764394, ..., 3269157, 2348354, 3055639])

Next, we compute the inputs and the resulting screening factors. For the screening pair, we select a random pair from amongst the possible options using `pynucastro.screening.get_screening_map`.

In [5]:
reaclib_library = pyna.ReacLibLibrary()

nuclei = [field[2:-1] for field in np.array(ds.field_list)[:,1] if "X(" in field]
comp = pyna.Composition(nuclei)

mynet = reaclib_library.linking_nuclei(comp.keys())
pynet = pyna.PythonNetwork(libraries=[mynet])

screen_map = pyna.screening.get_screening_map(
    pynet.get_rates(),
    symmetric_screening=pynet.symmetric_screening
)

In [6]:
pair = screen_map[15]
screening_kwargs = {
    "z1": pair.n1.Z,
    "a1": pair.n1.A,
    "z2": pair.n2.Z,
    "a2": pair.n2.A
}
screening_kwargs

{'z1': 2, 'a1': 4, 'z2': 18, 'a2': 36}

In [7]:
Xs = (np.array([ad[f"X({nucleus})"] for nucleus in nuclei]).T)
As = np.array(list(comp.A.values()))
Zs = np.array(list(comp.Z.values()))
Ys = Xs / As

In [8]:
D = np.array(ad["density"])
T = np.array(ad["Temp"])

abar = 1 / np.sum(Ys, axis=1)
zbar = np.sum(Zs * Ys, axis=1) * abar
z2bar = np.sum(Zs**2 * Ys, axis=1) * abar
log_z2bar = np.log10(z2bar)

z1 = screening_kwargs["z1"] * np.ones_like(T)
z2 = screening_kwargs["z2"] * np.ones_like(T)

In [9]:
F = chug.chugunov_2009.chugunov_2009(
    T=T, D=D,
    abar=abar, zbar=zbar, z2bar=z2bar,
    **screening_kwargs
)
actual_skip = F[choices] < 1.01

We find that screening can be skipped around half of the time, depending on our choices and the selected screening pair.

In [10]:
p = np.count_nonzero(actual_skip)
n = total - p
print(
    f"Can skip: {p}/{total} ({100 * p/total:.2f}%)",
    f"Can't skip: {n}/{total} ({100 * n/total:.2f}%)",
    sep="\n"
)

Can skip: 4802/10000 (48.02%)
Can't skip: 5198/10000 (51.98%)


From there, we use the interpolator and the power law described in prior notebooks to make our predictions.

In [11]:
screen_interp = DefaultScreeningData.default_interpolator
xi = np.array([abar[choices], log_z2bar[choices], z1[choices], z2[choices]]).T

In [12]:
C = screen_interp(xi, method="linear")
predict_skip = D[choices] < 1/10**C * T[choices]**3

In [13]:
TP = predict_skip & actual_skip
TN = ~predict_skip & ~actual_skip
FP = predict_skip & ~actual_skip
FN = ~predict_skip & actual_skip

tp = np.count_nonzero(TP)
tn = np.count_nonzero(TN)
fp = np.count_nonzero(FP)
fn = np.count_nonzero(FN)

A breakdown of the data is given below. To summarize, the predictions were correct the vast majority of the time, with false negatives making up a very small fraction of the total. False negatives are fine, as it just means the screening factor will end up being computed and found to be $1.01$ anyway. False positives are the real issue, as those mean that screening will be skipped even in a scenario where it could be relevant - but fortunately, there were no false positives.

In [14]:
print(
    f"True Positives: {tp}/{total} ({100 * tp/total:.2f}%)",
    f"True Negatives: {tn}/{total} ({100 * tn/total:.2f}%)",
    f"False Positives: {fp}/{total} ({100 * fp/total:.2f}%)",
    f"False Negatives: {fn}/{total} ({100 * fn/total:.2f}%)",
    sep="\n"
)

True Positives: 4795/10000 (47.95%)
True Negatives: 5198/10000 (51.98%)
False Positives: 0/10000 (0.00%)
False Negatives: 7/10000 (0.07%)


In [15]:
print(
    f"Accuracy: {100 * (tp + tn)/(p + n):.2f}%",
    f"Precision: {100 * tp/(tp + fp):.2f}%",
    f"Recall: {100 * tp/(tp + fn):.2f}%",
    sep="\n"
)

Accuracy: 99.93%
Precision: 100.00%
Recall: 99.85%
